# 0.0 IMPORTS

In [81]:
import pandas as pd 
import numpy as np 
import matplotlib as px
import seaborn as sns
import plotly as plt 
from sklearn.metrics.pairwise import haversine_distances
import math 
import haversine
import pygeodesic

In [75]:
!pip install pygeodesic

     ---------------------------------------- 1.3/1.3 MB 4.7 MB/s eta 0:00:00


# 0.1 Loading Data

In [2]:
df = pd.read_csv('./data/train.crdownload.csv')

## 1.0 DATA DESCRIPTIONS

In [3]:
df1 = df.copy()

In [4]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,19-03-2022,11:30:00,11:45:00,conditions Sunny,High,2,Snack,motorcycle,0,No,Urban,(min) 24
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,25-03-2022,19:45:00,19:50:00,conditions Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,(min) 33
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,19-03-2022,08:30:00,08:45:00,conditions Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,(min) 26
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,05-04-2022,18:00:00,18:10:00,conditions Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,(min) 21
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,26-03-2022,13:30:00,13:45:00,conditions Cloudy,High,1,Snack,scooter,1,No,Metropolitian,(min) 30


## 1.1 Data Dimensions

In [5]:
print(f'Number of Rows {df1.shape[0]}')
print(f'Number of Columns {df1.shape[1]}')

Number of Rows 4467
Number of Columns 20


## 1.2 Data Types

In [6]:
df1.sample(1)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
4038,0x9516,INDORES17DEL02,31,5,22.761226,75.887522,22.811226,75.937522,26-03-2022,20:10:00,20:20:00,conditions Sunny,Jam,2,Snack,electric_scooter,0,No,Urban,(min) 15


In [7]:
df1.dtypes

# 'Delivery_person_Age,Delivery_person_Ratings', Order_Date,Time_taken, multiple_deliveries,  #

ID                              object
Delivery_person_ID              object
Delivery_person_Age             object
Delivery_person_Ratings         object
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Order_Date                      object
Time_Orderd                     object
Time_Order_picked               object
Weatherconditions               object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries             object
Festival                        object
City                            object
Time_taken(min)                 object
dtype: object

In [8]:
df1.isnull().sum()

ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Orderd                    0
Time_Order_picked              0
Weatherconditions              0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           1
Time_taken(min)                1
dtype: int64

## 1.3 Change Types

In [9]:
# Delivery_person_Age
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].replace('NaN', 0)
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].str.strip()
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].replace('[^\d]+', '', regex=True)
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].fillna(0).astype('int64')



ValueError: invalid literal for int() with base 10: ''

In [ ]:
# Delivery_person_Age
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype('float64')

# Delivery_person_Age
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype('float64')

# Order_Date

df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], errors='coerce')
df1['Order_Date'] = df1['Order_Date'].dt.time

# # Time_taken 
# df1['Time_taken'] = pd.to_datetime(df1['Time_taken'], format='%H:%M:%S').dt.time

# multiple_deliveries

df1['multiple_deliveries'] = df1['multiple_deliveries'].astype('float64')

In [ ]:
df1.sample(5)

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min)
1178,0xc6bb,LUDHRES18DEL03,34,5.0,30.890184,75.829615,30.920184,75.859615,NaT,23:55:00,00:00:00,conditions Stormy,Low,1,Meal,scooter,1.0,No,Metropolitian,(min) 20
779,0xd5a6,DEHRES12DEL03,28,4.6,0.000000,0.000000,0.030000,0.030000,NaT,17:20:00,17:35:00,conditions Cloudy,Medium,2,Snack,scooter,1.0,No,Metropolitian,(min) 17
3885,0x8ba5,BANGRES05DEL02,27,5.0,12.970324,77.645748,13.050324,77.725748,NaT,19:50:00,20:05:00,conditions Windy,Jam,0,Buffet,motorcycle,1.0,No,Metropolitian,(min) 26
3657,0x8915,CHENRES17DEL02,23,4.7,13.045479,80.233110,13.125479,80.313110,NaT,20:35:00,20:50:00,conditions Windy,Jam,1,Snack,motorcycle,1.0,No,Metropolitian,(min) 22
4252,0xcb48,ALHRES04DEL01,26,4.4,0.000000,0.000000,0.110000,0.110000,NaT,21:40:00,21:55:00,conditions Cloudy,Jam,2,Snack,electric_scooter,1.0,No,Metropolitian,(min) 31


In [ ]:
df1.dtypes

ID                                     object
Delivery_person_ID                     object
Delivery_person_Age                    object
Delivery_person_Ratings               float64
Restaurant_latitude                   float64
Restaurant_longitude                  float64
Delivery_location_latitude            float64
Delivery_location_longitude           float64
Order_Date                     datetime64[ns]
Time_Orderd                            object
Time_Order_picked                      object
Weatherconditions                      object
Road_traffic_density                   object
Vehicle_condition                       int64
Type_of_order                          object
Type_of_vehicle                        object
multiple_deliveries                   float64
Festival                               object
City                                   object
Time_taken(min)                        object
dtype: object

## 1.4 Descriptive Statistical

In [ ]:
num_attributes = df1.select_dtypes('int64','float64')
cat_atribbutes = df1.select_dtypes('object','datetime64[ns]')

In [ ]:
# Central Tendency - Mean and Median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersion - Std, Min, Max, Range, Skew, Kurtosis
d1 = pd.DataFrame( num_attributes.apply( np.std)).T
d2 = pd.DataFrame( num_attributes.apply( min)).T
d3 = pd.DataFrame( num_attributes.apply( max)).T
d4 = pd.DataFrame( num_attributes.apply( lambda x: x.max() - x.min())).T
d5 = pd.DataFrame( num_attributes.apply( lambda x: x.skew())).T
d6 = pd.DataFrame( num_attributes.apply( lambda x: x.kurtosis())).T

# concatenate
m = pd.concat( [d2,d3,d4,ct1,ct2,d1,d5,d6]).T.reset_index()
m.columns = ['attributes','min','max','range','mean','median','std','skew','kurtosis']

m.reset_index()

,index,attributes,min,max,range,mean,median,std,skew,kurtosis
0,0,Vehicle_condition,0.0,3.0,3.0,1.022386,1.0,0.840246,0.077819,-1.309626


# 2 PERGUNTAS DE NEGOCIO

In [44]:
df1.nunique()

ID                             4467
Delivery_person_ID             1141
Delivery_person_Age              23
Delivery_person_Ratings          25
Restaurant_latitude             426
Restaurant_longitude            401
Delivery_location_latitude     2450
Delivery_location_longitude    2450
Order_Date                       44
Time_Orderd                     177
Time_Order_picked               193
Weatherconditions                 7
Road_traffic_density              5
Vehicle_condition                 4
Type_of_order                     4
Type_of_vehicle                   4
multiple_deliveries               5
Festival                          4
City                              4
Time_taken(min)                  45
dtype: int64

## 1. A quantidade de entregadores únicos.


In [55]:
delivery_unique = len ( df.loc[:, 'Delivery_person_ID'].unique() )

print(delivery_unique)


1141


## 2. A distância média dos resturantes e dos locais de entrega.

In [59]:
df1.dtypes

ID                              object
Delivery_person_ID              object
Delivery_person_Age             object
Delivery_person_Ratings         object
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Order_Date                      object
Time_Orderd                     object
Time_Order_picked               object
Weatherconditions               object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries             object
Festival                        object
City                            object
Time_taken(min)                 object
dtype: object

In [74]:

cols = ['Restaurant_latitude','Restaurant_longitude','Delivery_location_latitude','Delivery_location_longitude']

df_aux = df1.loc[:, cols].apply(lambda x: haversine(x['Restaurant_latitude'], x['Restaurant_longitude']), axis=1)

TypeError: 'module' object is not callable

In [72]:
df_aux

Restaurant_latitude            <module 'haversine' from 'c:\\Users\\rozen\\an...
Restaurant_latitude            <module 'haversine' from 'c:\\Users\\rozen\\an...
Delivery_location_latitude     <module 'haversine' from 'c:\\Users\\rozen\\an...
Delivery_location_longitude    <module 'haversine' from 'c:\\Users\\rozen\\an...
dtype: object